In [216]:
import pandas as pd
import numpy as np
wins = pd.read_csv('csv/sql/wins.csv')
team_games = pd.read_csv("csv/sql/team_games.csv", dtype= {'teamid': 'str'})

df = team_games.merge(wins, on="platformgameid")


df['win'] = np.where(df['side'] == df['winningteam'], 1, 0)
df.drop(columns=["winningteam"], inplace=True)
print(len(df))
display(df.head())

blue_wins = df[(df["win"] == 1) & (df["side"] == 100)]
red_wins = df[(df["win"] == 1) & (df["side"] == 200)]
print("Total Games: \t", len(df["esportsgameid"].unique()))
print("Blue wins: \t", len(blue_wins))
print("Red wins: \t", len(red_wins))
print("Blue win rate: \t", len(blue_wins)/len(wins))
print("Red win rate: \t", len(red_wins)/len(wins))

50462


,teamid,esportsgameid,platformgameid,side,win
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1
4,99566404856367466,110428848767875084,LPL_B:326759,200,1


Total Games: 	 25223
Blue wins: 	 13336
Red wins: 	 11895
Blue win rate: 	 0.5280538507226292
Red win rate: 	 0.4709958424074441


First Blood Data

In [217]:
first_blood = pd.read_csv("csv/sql/first_blood.csv")
first_blood = first_blood.sort_values(by=["platformgameid"])
first_blood = first_blood.drop_duplicates(subset=['platformgameid'], keep='first')
first_blood["gametime"] = first_blood["gametime"] / 60000
first_blood = first_blood.rename(columns={
    "gametime": "first_blood_time",
})
df = df.merge( first_blood, on="platformgameid" )
df['first_blood'] = np.where(df['side'] == df['killerteamid'], 1, 0)
df.drop(columns=["killerteamid"], inplace=True)

def wr_for_side(stat, side):
    side_name = "Blue" if side == 100 else "Red"
    blue_fb = df[(df[stat] == 1) & (df["side"] == side)]
    blue_fb_wins = df[(df[stat] == 1) & (df["side"] == side) & (df["win"] == 1)]
    print(f"{side_name} {stat}: \t", len(blue_fb))
    print(f"{side_name} {stat} wins: \t", len(blue_fb_wins))
    print(f"{side_name} {stat} win rate: \t", len(blue_fb_wins)/len(blue_fb))

wr_for_side(stat="first_blood", side=100)
wr_for_side(stat="first_blood", side=200)



Blue first_blood: 	 12661
Blue first_blood wins: 	 6739
Blue first_blood win rate: 	 0.5322644340889345
Red first_blood: 	 12499
Red first_blood wins: 	 5937
Red first_blood win rate: 	 0.4749979998399872


First Tower And INhibitor Data

In [218]:
first_building = pd.read_csv("csv/sql/first_buildings.csv")
first_building  = first_building[["platformgameid", "gametime", "teamid", "buildingtype"]]

first_tower = first_building[first_building["buildingtype"] == "turret"].copy()
first_tower = first_tower.sort_values(by=["platformgameid", "gametime"])
first_tower = first_tower.drop_duplicates(subset=['platformgameid'], keep='first')
# killer_team_id = first_tower["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_tower["first_tower_owner_id"] = first_tower["teamid"]
first_tower = first_tower.drop(columns=["teamid", "buildingtype"])
first_tower = first_tower.rename(columns={
    "gametime": "first_tower_time",
})
print("First tower: \t", len(first_tower))
display(first_tower.head())
first_inhibitor = first_building[first_building["buildingtype"] == "inhibitor"].copy()
first_inhibitor = first_inhibitor.sort_values(by=["platformgameid", "gametime"])
first_inhibitor = first_inhibitor.drop_duplicates(subset=['platformgameid'], keep='first')

first_inhibitor["first_inhibitor_owner_id"] = first_inhibitor["teamid"]
first_inhibitor = first_inhibitor.drop(columns=["teamid", "buildingtype"])
first_inhibitor = first_inhibitor.rename(columns={
    "gametime": "first_inhibitor_time",
})
print("First inhibitor: \t", len(first_inhibitor))
first_inhibitor.head()

First tower: 	 25252


,platformgameid,first_tower_time,first_tower_owner_id
40575,ESPORTSTMNT01:1110148,939441,200
23132,ESPORTSTMNT01:1110162,677086,200
31938,ESPORTSTMNT01:1110177,836608,200
12714,ESPORTSTMNT01:1110187,626524,100
39594,ESPORTSTMNT01:1110198,895424,200


First inhibitor: 	 25212


,platformgameid,first_inhibitor_time,first_inhibitor_owner_id
70824,ESPORTSTMNT01:1110148,1708748,200
41143,ESPORTSTMNT01:1110162,1602302,200
62898,ESPORTSTMNT01:1110177,1546286,200
3623,ESPORTSTMNT01:1110187,1628728,200
50722,ESPORTSTMNT01:1110198,1694081,200


In [219]:
df = df.merge(first_tower, on="platformgameid")
df = df.merge(first_inhibitor, on="platformgameid")
df['first_tower'] = np.where(df['side'] != df['first_tower_owner_id'], 1, 0)
df.drop(columns=["first_tower_owner_id"], inplace=True)
df['first_inhibitor'] = np.where(df['side'] != df['first_inhibitor_owner_id'], 1, 0)
df.drop(columns=["first_inhibitor_owner_id"], inplace=True)
df.head()

,teamid,esportsgameid,platformgameid,side,win,first_blood_time,first_blood,first_tower_time,first_inhibitor_time,first_tower,first_inhibitor
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1,8.489267,0,945005,1860636,1,1
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0,8.489267,1,945005,1860636,0,0
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0,4.042517,1,778010,1406628,0,0
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1,4.042517,0,778010,1406628,1,1
4,99566404856367466,110428848767875084,LPL_B:326759,200,1,4.330133,0,947875,1447813,1,1


In [220]:
wr_for_side(stat="first_tower", side=100)
wr_for_side(stat="first_tower", side=200)

Blue first_tower: 	 13837
Blue first_tower wins: 	 9632
Blue first_tower win rate: 	 0.6961046469610465
Red first_tower: 	 11353
Red first_tower wins: 	 7672
Red first_tower win rate: 	 0.6757685193340968


Kills and Deaths data

In [221]:
kills = pd.read_csv("csv/sql/kills.csv")
print(len(kills))
temp = df.merge(kills, on="platformgameid")

temp['kills'] = np.where(temp['side'] == 100, temp["blue_kills"], temp["red_kills"])
temp['deaths'] = np.where(temp['side'] == 100, temp["red_kills"], temp["blue_kills"])
temp.drop(columns=["blue_kills", "red_kills"], inplace=True)

df = temp.copy()

25252


In [222]:
first_four = pd.read_csv("csv/sql/first_four.csv")
print(len(first_four))
display(first_four.head())
temp = df.merge(first_four, on="platformgameid")

temp['level'] = np.where(temp['side'] == 100, temp["blue_level"], temp["red_level"])
temp['cs'] = np.where(temp['side'] == 100, temp["blue_cs"], temp["red_cs"])
temp.drop(columns=["blue_level", "red_level", "blue_cs", "red_cs"], inplace=True)

df = temp.copy()
display(df.head())

25240


,eventtime,platformgameid,gametime,winningteam,blue_level,red_level,blue_cs,red_cs
0,2022-03-02T19:15:03.099Z,ESPORTSTMNT01:2734334,210003,200,17,13,65.0,49.0
1,2021-12-22T10:16:45.215Z,ESPORTSTMNT02:2440617,210752,100,15,13,51.0,51.0
2,2019-07-05T11:11:20.733Z,ESPORTSTMNT03:1121320,210723,200,13,15,53.0,61.0
3,2023-02-12T18:16:06.723Z,ESPORTSTMNT01:3312264,210021,100,16,13,64.0,43.0
4,2022-09-04T10:16:32.977Z,ESPORTSTMNT02:2964845,210017,200,12,14,40.0,51.0


,teamid,esportsgameid,platformgameid,side,win,first_blood_time,first_blood,first_tower_time,first_inhibitor_time,first_tower,first_inhibitor,kills,deaths,eventtime,gametime,winningteam,level,cs
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1,8.489267,0,945005,1860636,1,1,19,9,2023-07-13T05:11:52.648Z,210019,200,18,64.0
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0,8.489267,1,945005,1860636,0,0,9,19,2023-07-13T05:11:52.648Z,210019,200,17,68.0
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0,4.042517,1,778010,1406628,0,0,5,28,2023-07-06T19:34:01.337Z,210030,100,16,62.0
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1,4.042517,0,778010,1406628,1,1,28,5,2023-07-06T19:34:01.337Z,210030,100,14,62.0
4,99566404856367466,110428848767875084,LPL_B:326759,200,1,4.330133,0,947875,1447813,1,1,24,6,2023-07-03T09:19:13.997Z,210022,200,17,60.0


In [223]:
event_start_times = pd.read_csv("csv/sql/event_start_times.csv")
event_start_times["event_start"] = pd.to_datetime(event_start_times["eventtime"], errors="coerce")
df = df.merge(event_start_times, on="platformgameid")
df.head()

,teamid,esportsgameid,platformgameid,side,win,first_blood_time,first_blood,first_tower_time,first_inhibitor_time,first_tower,first_inhibitor,kills,deaths,eventtime_x,gametime,winningteam,level,cs,eventtime_y,event_start
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1,8.489267,0,945005,1860636,1,1,19,9,2023-07-13T05:11:52.648Z,210019,200,18,64.0,2023-07-13T05:05:52.820Z,2023-07-13 05:05:52.820000+00:00
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0,8.489267,1,945005,1860636,0,0,9,19,2023-07-13T05:11:52.648Z,210019,200,17,68.0,2023-07-13T05:05:52.820Z,2023-07-13 05:05:52.820000+00:00
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0,4.042517,1,778010,1406628,0,0,5,28,2023-07-06T19:34:01.337Z,210030,100,16,62.0,2023-07-06T19:29:32.607Z,2023-07-06 19:29:32.607000+00:00
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1,4.042517,0,778010,1406628,1,1,28,5,2023-07-06T19:34:01.337Z,210030,100,14,62.0,2023-07-06T19:29:32.607Z,2023-07-06 19:29:32.607000+00:00
4,99566404856367466,110428848767875084,LPL_B:326759,200,1,4.330133,0,947875,1447813,1,1,24,6,2023-07-03T09:19:13.997Z,210022,200,17,60.0,2023-07-03T09:15:36.554Z,2023-07-03 09:15:36.554000+00:00


Rolling Averages And Exüprt

In [224]:
WINDOW = 5

df = df.sort_values(by=['teamid', 'event_start'])
# Group the DataFrame by 'team_id'
grouped = df.groupby('teamid')
# Calculate the rolling average for each group (team) using a window of 5
df['win_avg'] = grouped['win'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['kills_avg'] = grouped['kills'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['deaths_avg'] = grouped['deaths'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['level_avg'] = grouped['level'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['cs_avg'] = grouped['cs'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['first_blood_avg'] = grouped['first_blood'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['first_tower_avg'] = grouped['first_tower'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['first_inhibitor_avg'] = grouped['first_inhibitor'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['first_blood_time_avg'] = grouped['first_blood_time'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['first_tower_time_avg'] = grouped['first_tower_time'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)
df['first_inhibitor_time_avg'] = grouped['first_inhibitor_time'].rolling(window=WINDOW, min_periods=1).mean().reset_index(0, drop=True)


g2 = df[df["teamid"] =="98767991926151025"].sort_values(by=["event_start"])
display(g2.head(10))
display(g2[["platformgameid","first_blood","side","event_start", "first_blood_avg", "first_tower_avg", "first_inhibitor_avg"]].head(10))
df.to_csv("csv/sql/processed.csv", index=False)

,teamid,esportsgameid,platformgameid,side,win,first_blood_time,first_blood,first_tower_time,first_inhibitor_time,first_tower,...,kills_avg,deaths_avg,level_avg,cs_avg,first_blood_avg,first_tower_avg,first_inhibitor_avg,first_blood_time_avg,first_tower_time_avg,first_inhibitor_time_avg
35759,98767991926151025,101978171955957642,ESPORTSTMNT01:1120949,100,1,23.559817,0,796067,1573239,1,...,7.000000,3.000000,13.000000,54.000000,0.000000,1.000000,1.000000,23.559817,796067.000000,1573239.0
19654,98767991926151025,102072295521276197,ESPORTSTMNT01:1121076,200,0,4.567233,1,1001375,1611699,0,...,10.500000,16.500000,13.500000,56.500000,0.500000,0.500000,0.500000,14.063525,898721.000000,1592469.0
49347,98767991926151025,101978171954581382,ESPORTSTMNT01:1121126,100,1,8.933783,0,894966,1713465,0,...,12.666667,13.666667,14.333333,59.333333,0.333333,0.333333,0.666667,12.353611,897469.333333,1632801.0
25394,98767991926151025,101978171974635456,ESPORTSTMNT01:1121178,200,1,2.552767,0,947279,1438683,1,...,13.500000,13.250000,14.500000,61.000000,0.250000,0.500000,0.750000,9.903400,909921.750000,1584271.5
49680,98767991926151025,101978171956613004,ESPORTSTMNT01:1121276,200,0,1.875533,0,961405,1730802,0,...,12.200000,16.800000,14.800000,61.800000,0.200000,0.400000,0.600000,8.297827,920218.400000,1613577.6
16612,98767991926151025,101978171951828862,ESPORTSTMNT01:1121298,200,1,3.443967,0,1014716,2062671,0,...,15.400000,18.400000,15.000000,60.600000,0.200000,0.200000,0.600000,4.274657,963948.200000,1711464.0
38364,98767991926151025,101978171960807320,ESPORTSTMNT01:1121364,100,1,9.512883,1,900826,1616785,0,...,15.600000,13.800000,15.400000,59.600000,0.200000,0.200000,0.800000,5.263787,943838.400000,1712481.2
22003,98767991926151025,101978171977125831,ESPORTSTMNT01:1121389,100,0,5.626133,0,814874,1792591,0,...,15.800000,18.000000,15.400000,59.400000,0.200000,0.200000,0.800000,4.602257,927820.000000,1728306.4
43094,98767991926151025,101978171960479639,ESPORTSTMNT01:1121458,100,0,10.807167,1,855164,1848308,0,...,15.400000,19.800000,15.400000,58.000000,0.400000,0.000000,0.600000,6.253137,909397.000000,1810231.4
12236,98767991926151025,101978171950714747,ESPORTSTMNT01:1121470,200,0,3.869233,0,837618,1616015,0,...,15.200000,17.600000,15.200000,57.200000,0.400000,0.000000,0.600000,6.651877,884639.600000,1787274.0


,platformgameid,first_blood,side,event_start,first_blood_avg,first_tower_avg,first_inhibitor_avg
35759,ESPORTSTMNT01:1120949,0,100,2019-05-10 10:13:36.715000+00:00,0.000000,1.000000,1.000000
19654,ESPORTSTMNT01:1121076,1,200,2019-05-10 15:14:36.530000+00:00,0.500000,0.500000,0.500000
49347,ESPORTSTMNT01:1121126,0,100,2019-05-11 07:11:40.727000+00:00,0.333333,0.333333,0.666667
25394,ESPORTSTMNT01:1121178,0,200,2019-05-11 11:06:57.969000+00:00,0.250000,0.500000,0.750000
49680,ESPORTSTMNT01:1121276,0,200,2019-05-12 09:14:37.131000+00:00,0.200000,0.400000,0.600000
16612,ESPORTSTMNT01:1121298,0,200,2019-05-12 11:07:24.519000+00:00,0.200000,0.200000,0.600000
38364,ESPORTSTMNT01:1121364,1,100,2019-05-13 10:11:44.670000+00:00,0.200000,0.200000,0.800000
22003,ESPORTSTMNT01:1121389,0,100,2019-05-13 12:11:30.074000+00:00,0.200000,0.200000,0.800000
43094,ESPORTSTMNT01:1121458,1,100,2019-05-14 11:05:47.540000+00:00,0.400000,0.000000,0.600000
12236,ESPORTSTMNT01:1121470,0,200,2019-05-14 12:55:18.853000+00:00,0.400000,0.000000,0.600000
